## Setting up 

In [2]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool, CustomJS, LabelSet
import numpy as np

output_notebook()

Loading BokehJS ...

## Defining Variables and Functions

In [4]:
# long string variables
twentysix = [1,2,3,4,5,6,7,8,9,10,11,12,
             13,14,15,16,17,18,19,20,21,22,23,24,25,26]
alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l','m',
            'n','o','p','q','r','s','t','u','v','w','x','y','z']
alphabet = alphabet*3

# rotor shift
shift = 0

# aesthetic variables
x_offset = -4.5    # letter annotation offset from circle edge
y_offset = -7    # letter annotation offset from circle edge

seg_color = 'gold'    # segment color
line_width = 3.0    # segment line width
seg_alpha = 0.6    # segment alpha

circ_color = 'gold'    # circle color
circ_size = 23    # circle size
circ_alpha = 0.4    # circle alpha
hov_color = 'gold'    # hover color for circles
hov_alpha = 0.8    # hover alpha for circles

font = 'Garamond'


# some points
x = np.array([1,6,8])
x = np.repeat(x,26)
y = twentysix*3

In [5]:
# shifting functions

def shift_yvals(yvals, shift):
    for pair in yvals:
        pair[1] += shift
        if pair[1] > 26:
            pair[1] += (-26)
    return yvals

def shift_links(links,shift):
    for item in links:
        if item > 51:
            links[item][0] += (- shift)
            if links[item][0] < 26:
                links[item][0] += 26
        if item > 25 and item < 52:
            links[item][1] += shift
            if links[item][1] > 77:
                links[item][1] += (- 26)
    return(links)

## Creating X and Y values for paths

In [6]:
# X values
xval1 = [[1,6]]*26
xval2 = [[6,8]]*26
xvals = xval1+xval2

In [7]:
# Y values
# rotor 1
yvals_inner = [[1,5],[2,11],[3,13],[4,6],[5,12],[6,7],[7,4],[8,17],
    [9,22], [10,26],[11,14],[12,20],[13,15],[14,23],[15,25],[16,8],
    [17,24],[18,21],[19,19],[20,16],[21,1],[22,9],[23,2],[24,18],
    [25,3],[26,10]]
yvals_outer = [[1,1],[2,2],[3,3],[4,4],[5,5],[6,6],[7,7],[8,8],
    [9,9],[10,10],[11,11],[12,12],[13,13],[14,14],[15,15],[16,16],
    [17,17],[18,18],[19,19],[20,20],[21,21],[22,22],[23,23],[24,24],
    [25,25],[26,26]]

yvals_outer = shift_yvals(yvals_outer, shift)
yvals = yvals_inner + yvals_outer

# Rotor Links - Rotor 1

In [9]:
# establish links
links = {
    0:[30],      1:[36],     2:[38],     3:[31],      4:[37],    5:[32],     6:[29],     7:[42],     8:[47],    
    9:[51],     10:[39],    11:[45],    12:[40],    13:[48],    14:[50],    15:[33],    16:[49],    17:[46],    
    18:[44],    19:[41],    20:[26],    21:[34],    22:[27],    23:[43],    24:[28],    25:[35],
        
    26:[20,52],    27:[22,53],    28:[24,54],    29:[6,55],     30:[0,56],     31:[3,57],     32:[5,58],     
    33:[15,59],    34:[21,60],    35:[25,61],    36:[1,62],     37:[4,63],     38:[2,64],     39:[10,65],
    40:[12,66],    41:[19,67],    42:[7,68],     43:[23,69],    44:[18,70],    45:[11,71],    46:[17,72],
    47:[8,73],     48:[13,74],    49:[16,75],    50:[14,76],    51:[9,77],
    
    52:[26],    53:[27],    54:[28],    55:[29],    56:[30],    57:[31],    58:[32],    59:[33],    60:[34], 
    61:[35],    62:[36],    63:[37],    64:[38],    65:[39],    66:[40],    67:[41],    68:[42],    69:[43],
    70:[44],    71:[45],    72:[46],    73:[47],    74:[48],    75:[49],    76:[50],    77:[51]
}

links = shift_links(links,shift)

## Create Graph and Add Features

In [10]:
# create graph
p = figure(width=400, height=800, tools="",
           toolbar_location=None,title='Rotor I')

# add letter labels
lsource = ColumnDataSource(data=dict(x = x, y = y, names = alphabet))
labels = LabelSet(x='x', y='y', text='names', level='annotation',
                  x_offset=x_offset, y_offset=y_offset, source=lsource,
                  render_mode='canvas', text_font=font, 
                  text_color='black', text_font_style='bold')
p.add_layout(labels)

# add hover glyphs
source = ColumnDataSource({'x0': [], 'y0': [], 'x1': [], 'y1':[]})
sr = p.segment(x0='x0',y0='y0',x1='x1', y1='y1', color=seg_color, 
               alpha=seg_alpha, line_width=line_width, source=source,)
cr = p.circle(x, y, color=circ_color, size=circ_size, alpha=circ_alpha, 
              hover_color=hov_color, hover_alpha=hov_alpha)

# add path glyphs
p.multi_line(xvals,yvals,color='gray',alpha=0.3)

# remove gridlines and axis labels
p.xgrid.visible = False
p.ygrid.visible = False
p.axis.visible = False

# JS for hover tool
code = """
var links = %s;
var data = {'x0': [], 'y0': [], 'x1': [], 'y1': []};
var cdata = circle.get('data');
var indices = cb_data.index['1d'].indices;
for (i=0; i < indices.length; i++) {
    ind0 = indices[i]
    for (j=0; j < links[ind0].length; j++) {
        ind1 = links[ind0][j];
        data['x0'].push(cdata.x[ind0]);
        data['y0'].push(cdata.y[ind0]);
        data['x1'].push(cdata.x[ind1]);
        data['y1'].push(cdata.y[ind1]);
    }
}
segment.set('data', data);
""" % links

# Call JS and Add HoverTool
callback = CustomJS(args={'circle':cr.data_source, 
                          'segment':sr.data_source}, code=code)
p.add_tools(HoverTool(tooltips=None,callback=callback,renderers=[cr]))

# Display Graph

In [11]:
show(p)